### Callin Switzer

2 Nov 2016

-Read in all the Kalmia polygon images (from R KalmiaVisualizePolygons_V2.R)
- Make sure to use Python [conda env:vis_env]

-visualize in 3D

-Resample all of the images

-Stack resampled images (in 3D)

-Visualize a heatmap

-Save heatmap

In [1]:
import cv2

In [2]:
import numpy as np
import matplotlib
matplotlib.use("TkAgg") # have to use this for tkinter to  work below
from matplotlib import pyplot as plt
%matplotlib tk

In [3]:
# scikit image
import skimage
from skimage import io

In [4]:
import os
import pandas as pd
import time

In [5]:
import scipy

from mpl_toolkits.mplot3d import Axes3D
import sys

In [9]:
from mayavi import mlab
from scipy import stats

In [10]:
% qtconsole

May want to look at this: http://stackoverflow.com/questions/25286811/how-to-plot-a-3d-density-map-in-python-with-matplotlib

In [11]:
mu, sigma = 0, 0.1 
x = 10*np.random.normal(mu, sigma, 5000)
y = 10*np.random.normal(mu, sigma, 5000)
z = 10*np.random.normal(mu, sigma, 5000)

xyz = np.vstack([x,y,z])
kde = stats.gaussian_kde(xyz)
density = kde(xyz)

# Plot scatter with mayavi
figure = mlab.figure('DensityPlot')
pts = mlab.points3d(x, y, z, density, scale_mode='none', scale_factor=0.07)
mlab.axes()
mlab.show()

In [ ]:
# list files in directory

In [12]:
os.chdir("/Users/callinswitzer/Dropbox/KalmiaProject/KalmiaExamples/KalmiaManualTrig_VidsToProcess/")

In [13]:
# list files in directory
mypath = os.getcwd()
onlyfiles = [f for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))]
photoList = [ x for x in onlyfiles if not x.startswith('.') and x.endswith('.png')]

In [48]:
# read in image and resize
kk = 1
img = np.array(cv2.imread(photoList[kk], 0))

newSideLen = 100

r = float(newSideLen) / img.shape[1]
dim = (int(newSideLen), int(img.shape[0] * r))
 
# perform the actual resizing of the image and show it
resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

resized = np.array((resized < 255) * 1.0)

# clear edges of resized image
resized[:, 0] = 0
resized[newSideLen-1, :] = 0

plt.close('all')
io.imshow(resized)
thismanager = plt.get_current_fig_manager()
thismanager.window.wm_geometry("-1-1")




''

In [58]:
plt.imshow(resized)

In [63]:
# make empty np array
# for future reference, these are stacked wrong

# rotate image for visualization with numpy
resized_flip = np.rot90(resized, 3)
stackedArr = np.ones(shape=(np.repeat(newSideLen, [3]))) * 0.0

midpt = stackedArr.shape[1]/2

stackedArr[midpt,:,:] = stackedArr[midpt,:,:] + resized_flip

stackedArr[:, midpt, :] = stackedArr[:, midpt, :] + resized_flip

# get size
sys.getsizeof(stackedArr)/1000/1000 # in MB


0

In [64]:
x,y,z = stackedArr.nonzero()

In [65]:
#plt.close('all')
plt.clf()
fig = plt.figure(1)
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, z, zdir='z', c = stackedArr[x,y,z], 
           cmap = 'Greys', edgecolor = 'none', marker = 's')
#plt.savefig("demo.png")

In [66]:
plt.close('all')

In [67]:
# visualize with MayaVi
xyz = np.vstack([x, y, z,])
kde = stats.gaussian_kde(xyz)
density = kde(xyz)

# Plot scatter with mayavi
figure = mlab.figure('DensityPlot')
pts = mlab.points3d(x, y, z, stackedArr[x,y,z], colormap = 'cool', 
                    scale_mode='none', scale_factor=1)
mlab.axes()
mlab.show()

In [68]:
# visualize density with MayaVi
density = stackedArr
grid = mlab.pipeline.scalar_field(density)
min1 = density.min()
max1=density.max()
mlab.pipeline.volume(grid, vmin=min1, vmax=min1 + .5*(max1-min1))

mlab.axes()
mlab.show()

In [39]:
# Evaluate density on a grid
xmin, ymin, zmin = 0, 0, 0
xmax, ymax, zmax = newSideLen,newSideLen,newSideLen
xi, yi, zi = np.mgrid[xmin:xmax:30j, ymin:ymax:30j, zmin:zmax:30j]
density = stackedArr

In [44]:
grid = mlab.pipeline.scalar_field(density)

In [ ]:
# Plot scatter with mayavi
figure = mlab.figure('DensityPlot')

grid = mlab.pipeline.scalar_field(xi, yi, zi, density)

In [45]:
min = density.min()
max=density.max()
mlab.pipeline.volume(grid, vmin=min, vmax=min + .5*(max-min))

mlab.axes()
mlab.show()

In [ ]:
# plot 3d points with mlab
mlab.points3d(x,y,z)

In [ ]:
# plot stacked array
mlab.contour3d(stackedArr, colormap = 'jet')

In [ ]:
mlab.pipeline.volume(stackedArr)

In [ ]:
from scipy.interpolate import griddata

# Create some test data, 3D gaussian, 200 points
dx, pts = 2, 100j

N = 500
R = np.random.random((N,3))*2*dx - dx
V = np.exp(-( (R**2).sum(axis=1)) )

# Create the grid to interpolate on
X,Y,Z = np.mgrid[-dx:dx:pts, -dx:dx:pts, -dx:dx:pts]

# Interpolate the data
F = griddata(R, V, (X,Y,Z))
mlab.contour3d(F,contours=8,opacity=.2 )


In [ ]:
mlab.clf()
mu, sigma = 0, 0.1 
x = 10*np.random.normal(mu, sigma, 5000)
y = 10*np.random.normal(mu, sigma, 5000)    
z = 10*np.random.normal(mu, sigma, 5000)

xyz = np.vstack([x,y,z])
kde = stats.gaussian_kde(xyz)

# Evaluate kde on a grid
xmin, ymin, zmin = x.min(), y.min(), z.min()
xmax, ymax, zmax = x.max(), y.max(), z.max()
xi, yi, zi = np.mgrid[xmin:xmax:30j, ymin:ymax:30j, zmin:zmax:30j]
coords = np.vstack([item.ravel() for item in [xi, yi, zi]]) 
density = kde(coords).reshape(xi.shape)

# Plot scatter with mayavi
figure = mlab.figure('DensityPlot')

grid = mlab.pipeline.scalar_field(xi, yi, zi, density)
min = density.min()
max=density.max()
mlab.pipeline.volume(grid, vmin=min, vmax=min + .5*(max-min))

mlab.axes()
mlab.show()

In [ ]:
plt.close('all')
verts, faces = measure.marching_cubes(stackedArr, 0, spacing=(0.1, 0.1, 0.1))

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_trisurf(verts[:, 0], verts[:,1], faces, verts[:, 2], facecolors = ['red', 'blue'],
                cmap='Purples', lw=5, alpha = 0.2, edgecolor = 'none')
plt.show()



In [ ]:
plt.clf()
io.imshow(resized)

In [ ]:
# rotate image
M = cv2.getRotationMatrix2D(center, 180, 1.0)
rotated = cv2.warpAffine(image, M, (w, h))
cv2.imshow("rotated", rotated)

In [ ]:
plt.clf()
i2 = scipy.ndimage.interpolation.rotate(np.array(resized), 33, axes=(1,0))
io.imshow(i2)

In [ ]:
# visualize individual polygons without weighting
# in 3D

kk = 0
fpth = photoList[kk]
img = cv2.imread(fpth,0)
img = (img < 255) * 1.0

# these are the points of interest
intPoints = np.array([[768, 1694],[1000, 1694],[1231, 1694]])

#img[:, 1000]= 0.9
#img[1694, :] = 0.9
plt.clf()
io.imshow(img, cmap = 'gray')
plt.scatter(x = intPoints[:,0], y = intPoints[:,1])



# make multiplication matrix
mask = np.ones((2000, 2000))

# make a matrix to multiply by
mask2 = mask 
io.imshow(mask2 * img, cmap = 'gray')

In [ ]:
img = cv2.imread(fpth, 0)
img = img / np.max(img)
x, y = np.ogrid[0:img.shape[0], 0:img.shape[1]]
ax = plt.gca(projection='3d')
ax.plot_surface(x, y, 10, rstride=5, cstride=5, facecolors=img)
plt.show()

In [ ]:
plt.clf()
plt.hist(np.ndarray.flatten(stackedImg))
plt.ylim([0, 500])

In [ ]:
# stack all polygons and visualize
stackedImg = np.ones((2000, 2000)) * 0.0



for ii in range(len(photoList)):
    # read in image
    
    fpth = photoList[ii]
    img = cv2.imread(fpth,0)
    img = (img < 255) * 1.0
    if ii == 0:
        img = cv2.flip(img, 1)
    img2 = mask2 * img
    stackedImg = img2 + stackedImg
    
plt.clf()
io.imshow(stackedImg)

In [ ]:
# resample polygons and flip random ones

io.imshow(cv2.flip(img, 1))

In [ ]:
def resampleImgs():
    # stack all polygons and visualize
    stackedImg = np.ones((2000, 2000)) * 0.0

    # make multiplication matrix
    mask = np.ones((2000, 2000))

    # make points exponentially decreasing 
    # this makes the center probability  == 10 times the outside probability
    row1 = (10.0**((np.hstack([range(0, 1000), range(999, -1, -1)]))/231.0))

    # normalize
    row1 = row1 / np.max(row1)
    mask2 = mask * (row1)




    #sample photolist
    photoSamp = np.random.choice(range(len(photoList)), size=len(photoList), replace=True, p=None)


    for ii in photoSamp:
        # read in image
        fpth = photoList[ii]
        img = cv2.imread(fpth,0)
        img = (img < 255) * 1.0



        #flip coin for whether or not to flip image
        flip = np.random.choice([True, False], size=1, replace=True, p=None)
        if flip:
            img = cv2.flip(img, 1)

        img2 = mask2 * img

        # stack images
        stackedImg = img2 + stackedImg
    
    return stackedImg

In [ ]:
imm = resampleImgs()
plt.clf()
plt.hist(np.ndarray.flatten(imm))
plt.ylim([0, 5000])

In [ ]:
plt.clf()
io.imshow(imm)

In [ ]:
stt = time.time()
imStt = np.array([resampleImgs() for ii in range(100)])
print time.time() - stt

In [ ]:
# sum up arrays to make total bootstrap resample 
imSum = imStt.sum(axis = 0)

# normalize
imSum = imSum / np.max(imSum)

plt.clf()
io.imshow(imSum, cmap = 'magma')
plt.xlim([600, 1400])
plt.ylim([ 1700, 700])

In [ ]:
plt.clf()
io.imshow(imSum, cmap = 'YlOrRd')
plt.xlim([600, 1400])
plt.ylim([ 1700, 700])

# add points
plt.axis([0, imSum.shape[1], imSum.shape[0], 0])
plt.axis('off')

plt.gca().set_position([0,0,1,1])
plt.gca().set_aspect('equal')

intPoints = np.array([[768, 1694],[1000, 1694],[1231, 1694]])
plt.scatter(x = intPoints[:,0], y = intPoints[:,1])
plt.xlim([600, 1400])
plt.ylim([ 1800, 700])

plt.show()

plt.savefig('PollenHeatmap_Yellow.png')

In [ ]:
plt.clf()
io.imshow(imSum, cmap = 'Purples')
plt.xlim([600, 1400])
plt.ylim([ 1700, 700])

# add points
plt.axis([0, imSum.shape[1], imSum.shape[0], 0])
plt.axis('off')

plt.gca().set_position([0,0,1,1])
plt.gca().set_aspect('equal')

intPoints = np.array([[768, 1694],[1000, 1694],[1231, 1694]])
plt.scatter(x = intPoints[:,0], y = intPoints[:,1])
plt.xlim([600, 1400])
plt.ylim([ 1800, 700])

plt.show()

plt.savefig('PollenHeatmap_Purples.png')

In [ ]:
plt.clf()
io.imshow(imSum, cmap = 'hot')
plt.xlim([600, 1400])
plt.ylim([ 1700, 700])

# add points
plt.axis([0, imSum.shape[1], imSum.shape[0], 0])
plt.axis('off')

plt.gca().set_position([0,0,1,1])
plt.gca().set_aspect('equal')

intPoints = np.array([[768, 1694],[1000, 1694],[1231, 1694]])
plt.scatter(x = intPoints[:,0], y = intPoints[:,1])
plt.xlim([600, 1400])
plt.ylim([ 1800, 700])

plt.show()

plt.savefig('PollenHeatmap_Hot.png')

In [ ]:
plt.clf()
io.imshow(imSum, cmap = 'magma')
plt.xlim([600, 1400])
plt.ylim([ 1700, 700])

# add points
plt.axis([0, imSum.shape[1], imSum.shape[0], 0])
plt.axis('off')

plt.gca().set_position([0,0,1,1])
plt.gca().set_aspect('equal')

intPoints = np.array([[768, 1694],[1000, 1694],[1231, 1694]])
plt.scatter(x = intPoints[:,0], y = intPoints[:,1])
plt.xlim([600, 1400])
plt.ylim([ 1800, 700])

plt.show()

plt.savefig('PollenHeatmap.png')

In [ ]:
plt.clf()
io.imshow(imSum, cmap = 'gray_r')
plt.xlim([600, 1400])
plt.ylim([ 1700, 700])

# add points
#plt.axis([0, imSum.shape[1], imSum.shape[0], 0])
#plt.axis('off')

#plt.gca().set_position([0,0,1,1])
#plt.gca().set_aspect('equal')

intPoints = np.array([[768, 1694],[1000, 1694],[1231, 1694]])
plt.scatter(x = intPoints[:,0], y = intPoints[:,1])
plt.xlim([600, 1400])
plt.ylim([ 1800, 700])

plt.show()

#plt.savefig('PollenHeatmap_BW.png')

In [ ]:
plt.clf()
plt.hist(np.ndarray.flatten(imSum), bins = 100)
plt.ylim([0, 5000])

In [ ]:
# plot with colorbar

plt.close('all')

fig, ax = plt.subplots()
im = ax.imshow(imSum, cmap = 'gray_r')
cax = fig.add_axes([0.78, 0.2, 0.05, 0.6])
fig.colorbar(im, cax=cax, orientation='vertical')
# plt.xlim([600, 1400])
# plt.ylim([ 1700, 700])

ax.axis('off')

ax.set_xlim([600, 1400])
ax.set_ylim([ 1800, 700])

intPoints = np.array([[768, 1694],[1000, 1694],[1231, 1694]])
ax.scatter(x = intPoints[:,0], y = intPoints[:,1])

plt.show()
plt.savefig('PollenHeatmap_BW_colorbar.png')

In [ ]:
data = np.arange(100, 0, -1).reshape(10, 10)

fig, ax = plt.subplots()
#cax = fig.add_axes([0.27, 0.8, 0.5, 0.05])

im = ax.imshow(data, cmap='gist_earth')
fig.colorbar(im, cax=cax, orientation='horizontal')
plt.show()